In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import platform
print(platform.platform())
import torch
print(torch.__version__)
!nvcc --version

Fri Oct 20 09:03:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0  On |                  Off |
|  0%   35C    P8              10W / 500W |    326MiB / 24564MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

C:\Users\rvc_sw_mss1_common\Desktop\test\venv\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\rvc_sw_mss1_common\Desktop\test\venv\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\rvc_sw_mss1_common\Desktop\test\venv\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


2.0.1+cu118
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
prompt = """Use the following pieces of information to answer the instruction at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. You should only answer once and do not give any extra text after the answer is done.
{input}
{history}
### Instruction: {question}
### Response:"""

prompt_for_common_query =  """### Instruction: AI Chat Assistant
You are an AI Chat assistant. User will talk to you or give you a question. Your goal is to response politely to user. Just respond and do not give any extra information:
<history>
User: {prev_question} <|end_of_turn|>
Assistant: {prev_answer} <|end_of_turn|>
</history>
Information: {input}
Question: {question}
End your response with <|end_of_turn|>
### Response:
"""

prompt_for_specific_inquiry = """### Instruction: AI Document Assistant
You are a helpful AI Document assistant and you can understand any long document. Your goal is to obtain the following pieces of information to get your understand. Then answer the question at the end of the information pieces.
If you need more information in the previous context, please refer to the 'history' part. Consider the previous chat of this conversation:
<history>
User: {prev_question} <|end_of_turn|>
Assistant: {prev_answer} <|end_of_turn|>
</history>
Now read the following information pieces and answer the question:
Information: {input}
Question: {question}
End your response with <|end_of_turn|>
### Response:
"""

prompt_for_followup_question = """System: <<SYS>>You are a helpful assistant, you always only answer for the assistant then you stop. Do not self-create human message and give extra assistant response turn. Now read the chat history to get context<</SYS>>
User: {prev_question} <|end_of_turn|>
Assistant: {prev_answer} <|end_of_turn|>
User: {current_question}
"""


# prompt = """You are a helpful and honest AI assistant. Using the given pieces of Clue Information to answer the Instruction at the end of the prompt. You should only answer once and do not give any extra text after the answer is done.
# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer, please don't try to make up an answer.
# CLUE INFORMATION:
# {input}
# {history}
# ### Instruction: {question}"""


relate_question_prompt = '''### Instruction: You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can.
Task: Generate at least 2 questions that related to the below conversation.
Question: {question}
Answer: {answer}
End your response with <|end_of_turn|>
### Response:
'''

# **DEFINE MODEL**

## Define

In [3]:
model_dir = "C:\\Dev_Env\HyLa\\source_code\\AI_MCAL\\model\\OpenOrca-Platypus2-13B-GPTQ"
print(model_dir)

C:\Dev_Env\HyLa\source_code\AI_MCAL\model\OpenOrca-Platypus2-13B-GPTQ


In [4]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

device = "cuda:0"

quantize_config = BaseQuantizeConfig(
    bits=4,  # quantize model to 4-bit
    #damp_percent= 0.01,
    desc_act= False,
    group_size=128,  # it is recommended to set the value to 128
)

#model = AutoGPTQForCausalLM.from_quantized(pretrained_model_name_or_path=model_dir,
                                            #device="cuda:0")

model = AutoGPTQForCausalLM.from_quantized(
    model_dir,
    # strict=False,
    use_triton=False,
    map_device='auto',
    use_safetensors=True,
    quantize_config=quantize_config,
    model_basename='model'
)

tokenizer = AutoTokenizer.from_pretrained(model_dir, map_device='auto')

  0%|          | 0/1483 [00:00<?, ?w/s]

skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [5]:
import gc
import torch
def clear_torch_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [6]:
def decode(output_ids, skip_special_tokens=True):
    return tokenizer.decode(output_ids, skip_special_tokens)

def get_reply_from_output_ids(output_ids, input_ids):
    new_tokens = len(output_ids) - len(input_ids[0])
    reply = decode(output_ids[-new_tokens:], skip_special_tokens = True)

    # Prevent LlamaTokenizer from skipping a space
    if len(output_ids) > 0:
        if tokenizer.convert_ids_to_tokens(int(output_ids[-new_tokens])).startswith('▁'):
            reply = ' ' + reply

    return reply

In [7]:
def model_generate(prompt_template):
  clear_torch_cache()
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
  clear_torch_cache()
  return tokenizer.decode(output[0])

In [8]:
clear_torch_cache()

In [9]:
max_input_size = model.config.max_position_embeddings
print("Maximum input size:", max_input_size)

Maximum input size: 4096


In [10]:
def extract_model_text(text:str):
  start_index = text.find("### Response:") + len("### Response:")
  end_index = text.find("</s>", start_index)

  # Extract the response text
  response_text = text[start_index:end_index].strip()
  return response_text

## Inferences

### Classify Sentences

In [11]:
import re
classification_prompt_template_1 = '''### Instruction: Sentence Classification
You are an AI assistant that could classify sentence. You will be given a sentence. Your task is to determine if a sentence is an ongoing sentence or not.
- "ongoing" are follow-up sentences need to refer back to a previous topic or seeks further information based on the previous context. It often include pronouns words that indicate to the previous subject like (it, those, these, they,...).
If the sentence is follow-up, answer "yes". Else answer "no".  Do Not try to explain anything.
Now please determine the below sentence:
Sentence: {sentence}
### Response:
'''

def extract_sentence_type_1(text:str):
  patternlist = [r"yes", r"no"]
  typelist = []
  for pattern in patternlist:
    match = re.search(pattern, text)
    if match:
      typelist.append(match.group())

  return typelist

def model_classify_question_1(sentence:str):
  # Replace the {question} placeholder with the provided question
  clone_template = classification_prompt_template_1
  used_prompt = clone_template.format(sentence=sentence)
  model_response = model_generate(used_prompt)
  question_type = extract_model_text(model_response)
  question_type_list = extract_sentence_type_1(question_type.lower())
  #print(f"[DEBUG] question_type_list: {question_type_list}")
  if "no" in question_type_list:
    return False
  elif "yes" in question_type_list:
    return True
  else:
    print("unknown answer")
    return False


# **INSTALL LIBRARY**

In [12]:
from langchain.chains import RetrievalQA, LLMChain, ConversationalRetrievalChain
from langchain.llms import OpenAI, HuggingFacePipeline
from langchain.document_loaders import TextLoader, PDFMinerPDFasHTMLLoader, PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.retrievers import SVMRetriever, TFIDFRetriever
from sentence_transformers import SentenceTransformer
import time
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# **TEXT GENERATION CODE**

## Funciton Init

In [45]:
streaming_text_test = """
    def generateAnswer(title:str, conversation:list, session:gr.State):
    Send question to model server and get streammed data from it.
    print("\n\n##########################################################")
    print("[Web-Server]______ generateAnswer")
    
    # Check for valid Chat mode
    if "chat_mode" in session and session["chat_mode"] == ChatMode.GENERAL_CHAT: 
        # Check for valid conversation with valid question
        if len(conversation) > 0 and len(conversation[-1]) > 0 and isinstance(conversation[-1][0], str) and len(conversation[-1][0]) > 0 and (conversation[-1][1] == None or len(conversation[-1][1]) <= 0): 
            question = conversation[-1][0] # Get question from the latest chat
        
            # Clone conversation content from session if available
            if "content" in session: chatHistoryContents = session["content"]
            else: chatHistoryContents = {}

            # Get streammed data from Model Server
            answer = ""
            link_buf = ""
            for streamStatus, ans, key in modelController.streamResponse(ChatMode.GENERAL_CHAT.value, session["token"], question, conversation):
                if streamStatus == Status.SUCCESS:
                    session["streaming_key"] = key
                    answer, currentRelatedQuestion, link_buf = extract_link(latest_ans=answer, current_ans=ans, link_buf=link_buf)
                    return@
                    def generateAnswer(title:str, conversation:list, session:gr.State):
    Send question to model server and get streammed data from it.
    print("\n\n##########################################################")
    print("[Web-Server]______ generateAnswer")
    
    # Check for valid Chat mode
    if "chat_mode" in session and session["chat_mode"] == ChatMode.GENERAL_CHAT: 
        # Check for valid conversation with valid question
        if len(conversation) > 0 and len(conversation[-1]) > 0 and isinstance(conversation[-1][0], str) and len(conversation[-1][0]) > 0 and (conversation[-1][1] == None or len(conversation[-1][1]) <= 0): 
            question = conversation[-1][0] # Get question from the latest chat
        
            # Clone conversation content from session if available
            if "content" in session: chatHistoryContents = session["content"]
            else: chatHistoryContents = {}

            # Get streammed data from Model Server
            answer = ""
            link_buf = ""
            for streamStatus, ans, key in modelController.streamResponse(ChatMode.GENERAL_CHAT.value, session["token"], question, conversation):
                if streamStatus == Status.SUCCESS:
                    session["streaming_key"] = key
                    answer, currentRelatedQuestion, link_buf = extract_link(latest_ans=answer, current_ans=ans, link_buf=link_buf)
                    return@
                    def generateAnswer(title:str, conversation:list, session:gr.State):
    Send question to model server and get streammed data from it.
    print("\n\n##########################################################")
    print("[Web-Server]______ generateAnswer")
    
    # Check for valid Chat mode
    if "chat_mode" in session and session["chat_mode"] == ChatMode.GENERAL_CHAT: 
        # Check for valid conversation with valid question
        if len(conversation) > 0 and len(conversation[-1]) > 0 and isinstance(conversation[-1][0], str) and len(conversation[-1][0]) > 0 and (conversation[-1][1] == None or len(conversation[-1][1]) <= 0): 
            question = conversation[-1][0] # Get question from the latest chat
        
            # Clone conversation content from session if available
            if "content" in session: chatHistoryContents = session["content"]
            else: chatHistoryContents = {}

            # Get streammed data from Model Server
            answer = ""
            link_buf = ""
            for streamStatus, ans, key in modelController.streamResponse(ChatMode.GENERAL_CHAT.value, session["token"], question, conversation):
                if streamStatus == Status.SUCCESS:
                    session["streaming_key"] = key
                    answer, currentRelatedQuestion, link_buf = extract_link(latest_ans=answer, current_ans=ans, link_buf=link_buf)
                    return@
                    """

In [46]:
import gc
import traceback
from queue import Queue
from threading import Thread
import torch
import transformers
from transformers import GenerationConfig, TextIteratorStreamer

def clear_torch_cache():
    gc.collect()
    torch.cuda.empty_cache()

class Stream(transformers.StoppingCriteria):
    """ Used to detect specific stopping conditions during the generation process.
    The Stream class takes a callback_func argument, which is a callback function to be called when
    a specific stopping condition is met during the generation process.
    """
    def __init__(self, callback_func=None):
        self.callback_func = callback_func

    def __call__(self, input_ids, scores) -> bool:
        if self.callback_func is not None:
            self.callback_func(input_ids[0])
        return False

class Iteratorize:

    """
    Transforms a function that takes a callback
    into a lazy iterator (generator).
    It wraps the function call in a separate thread, allowing the generator to run asynchronously
    and produce output as it becomes available.
    Adapted from: https://stackoverflow.com/a/9969000
    """

    def __init__(self, func, kwargs=None, callback=None):
        self.mfunc = func
        self.c_callback = callback
        self.q = Queue()
        self.sentinel = object()
        self.kwargs = kwargs or {}
        self.stop_now = False

        #print("[iteratorize_init]")
        def _callback(val):
            self.q.put(val)

        def gentask():
            print(f"[DEBUG] Iteratorize gentask called")
            try:
                print(f"[DEBUG] Iteratorize gentask await mfunc")
                ret = self.mfunc(callback=_callback, **self.kwargs)

            except ValueError:
                print(f"[DEBUG] Iteratorize gentask value error")
                pass
            except:
                print(f"[DEBUG] Iteratorize gentask traceback")
                traceback.print_exc()
                pass

            clear_torch_cache()
            self.q.put(self.sentinel)
            if self.c_callback:
                self.c_callback(ret)

        #gentask()
        self.thread = Thread(target=gentask)
        self.thread.start()
        print(f"[DEBUG] Iteratorize __init__ called")

    def __iter__(self):
        print(f"[DEBUG] Iteratorize __iter__ called")
        return self

    def __next__(self):
        obj = self.q.get(True, None)
        #print(f"[DEBUG] Iteratorize __next__ called")
        #print(f"[iteratorize_next] obj: {obj}") 
        if obj is self.sentinel:
            #print("[DEBUG] Iteratorize raise")
            raise StopIteration
        else:
            #print("[DEBUG] Iteratorize return")
            return obj
    
    def __del__(self):
        clear_torch_cache()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop_now = True
        clear_torch_cache()

def decode(output_ids, skip_special_tokens=True):
    return tokenizer.decode(output_ids, skip_special_tokens)

def get_reply_from_output_ids(output_ids, input_ids):
    new_tokens = len(output_ids) - len(input_ids[0])
    reply = decode(output_ids[-new_tokens:], skip_special_tokens = True)

    # Prevent LlamaTokenizer from skipping a space
    if len(output_ids) > 0:
        if tokenizer.convert_ids_to_tokens(int(output_ids[-new_tokens])).startswith('▁'):
            reply = ' ' + reply

    return reply

In [47]:
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

## ChatNew Function

In [92]:
def chatNew(prompt, related_question_prompt, references:list, wkproduct_links:list, guideline_links:list, guidelines:list):
  print("Start generating ----------------")
  result = ""
  stop_words = ["### Human", "Unhelpful answer:", "#include", "using namespace std;", "##", "import {"]

  # Iterate over each stop word in stop_words and converts each stop word to its corresponding token IDs using the tokenizer.
  # The return_tensors='pt' argument indicates that the tokenizer should return PyTorch tensors.
  # The squeeze() method is used to remove any extra dimensions from the tensor.
  stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]

  # Used to define stopping criteria for the model during generation.
  stopping_criteria_list = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

  # Tokenize the input prompt and convert it to PyTorch tensors.
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids

  # Move the input tensor to the GPU (if available).
  input_ids = input_ids.to('cuda')

  # Define a list of end-of-sentence (EOS) token IDs
  # The eos_token_id is the token ID of the end-of-sentence token used by the tokenizer.
  eos_token_ids = [tokenizer.eos_token_id] if tokenizer.eos_token_id is not None else []
  streamer = TextIteratorStreamer(tokenizer)
  # Dictionary holds various parameters for the model generation
  generate_params = {}
  generate_params['inputs'] = input_ids
  generate_params['max_new_tokens'] = 512
  generate_params['temperature'] = 0.7
  generate_params['top_p'] = 0.95
  generate_params['repetition_penalty'] = 1.15
  generate_params['eos_token_id'] = eos_token_ids
  generate_params['stopping_criteria'] = stopping_criteria_list

  def generate_with_callback(callback=None, **kwargs):
      """The function is defined to execute the model generation with a callback function.
      \n\tThe Stream instance is added to the stopping_criteria list to enable the callback.
        """
      print(f"[DEBUG] Generating with callback...")
      kwargs['stopping_criteria'].append(Stream(callback_func=callback))
      clear_torch_cache()
      #with torch.no_grad():
          #loop = asyncio.get_event_loop()
          #task = loop.create_task(model.generate, kwargs)
          #await task
      with torch.no_grad():
          model.generate(**kwargs)
          
  iterator = Iteratorize(generate_with_callback, generate_params)
  #iterator = generate_with_callback(generate_params)
  # Iterate over the generator and print the output
  for output in iterator:
      result = get_reply_from_output_ids(output, input_ids)
      #print(f"[DEBUG] chatNew() - result: {result}")
      yield result
      if output[-1] in eos_token_ids:
        break

  print("DONE GENERATE RESULT +++++++++++++++")

  # Generate references and related questions if available
  # If these are not any related question, the current query is a common chat
  if related_question_prompt != " ":
    related_question_prompt = related_question_prompt.replace('{answer}', result)
    print("\n############## QUESTION PROMPT ##############")
    print(related_question_prompt)
    print("+++++++++++++++++++++++++++++++++++++++")
    # Initialize a dictionary to store link types and their corresponding lists
    data_dict = {
        "Work Product links": wkproduct_links,
        "Guideline links": guideline_links,
        "Guideline" : guidelines,
        "Reference documents": references
    }

    # Iterate through the link types in the dictionary
    for data_type, data_list in data_dict.items():
        # Check if the data list is not empty
        if data_list is not None and len(data_list) > 0:
            # Generate the header for the data type
            result = f"\n**{data_type}:**\n"
            # Generate the data with yield in for loop
            for data in data_list:
                result += data + "\n"
            result += "<br>."
            # Yield the result
            yield result
      
    # Generate related questions
    print("START GENERATE RELATED QUESTION ++++++")
    result = "\n **Related question:**\n"
    input_ids = tokenizer(related_question_prompt, return_tensors='pt').input_ids
    input_ids = input_ids.to('cuda')
    eos_token_ids = [tokenizer.eos_token_id] if tokenizer.eos_token_id is not None else []
    generate_params['inputs'] = input_ids
    generate_params['eos_token_id'] = eos_token_ids
    generate_params['stopping_criteria'] = stopping_criteria_list

    iterator = Iteratorize(generate_with_callback, generate_params)
    # Iterate over the generator and print the output
    for output in iterator:
        complete_result = result + get_reply_from_output_ids(output, input_ids)
        yield complete_result
        #yield result + tokenizer.decode(output[0])
        if output in stop_words:
            break

In [109]:
#def chatNew(prompt, related_question_prompt, references:list, wkproduct_links:list, guideline_links:list, guidelines:list):
#  print("Start generating ----------------")
#  
#  for output in streaming_text_test:
#      yield output
#
#  print("DONE GENERATE RESULT +++++++++++++++")
!pip install redis

  Obtaining dependency information for redis from https://files.pythonhosted.org/packages/0b/34/a01250ac1fc9bf9161e07956d2d580413106ce02d5591470130a25c599e3/redis-5.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/250.3 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.3 kB 1.3 MB/s eta 0:00:01
   ---- ----------------------------------- 30.7/250.3 kB 1.3 MB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.3 kB 326.8 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.3 kB 581.0 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.3 kB 726.2 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.3 kB 701.4 kB/s eta 0:00:01
   -------------------------------------  245.8/250.3 kB 942.1 kB/s eta 0:00:01
   -------------------------------------- 250.3/250.3 kB 902.4 kB/s eta 0:00:00


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **Server**

In [253]:
import sys
import os
# Get the current working directory
current_dir = os.path.dirname(os.path.realpath('MCAL_BOT.ipynb'))
print(current_dir)

# Add the parent folder to the system path
QnA_module_path = os.path.join(current_dir, '..\\..\\03_Prompt') 
sys.path.append(QnA_module_path)


C:\Users\rvc_sw_mss1_common\Desktop\test\mcal-guru\MCAL_BOT_Chatbot\ModelServer


In [254]:
##########################################################
# Server Import Packages and Dependencies
##########################################################
import sys
sys.path.append('03_Prompt/')

import traceback
from queue import Queue
from flask import Flask, request, Response, jsonify, stream_with_context, g, session
from flask_session import Session
import atexit
import asyncio
import json
import time
import requests
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from _QnA import QnA
import threading
import concurrent.futures
import queue
from typing import Tuple
import pickle
from pympler import asizeof
import mysql.connector

In [260]:
MAX_CONCURRENCY = 16
semaphore = asyncio.Semaphore(MAX_CONCURRENCY)
session_semaphore = asyncio.Semaphore(MAX_CONCURRENCY)
global_counter = 0
support_sessions = {}
support_sessions_lock = threading.Lock()

In [261]:
app = Flask(__name__)
app.secret_key = 'af84e3cdeaed21a9220fb4fb7a9611de9d1abf85e0642ef50c71076a9fcba177'
app.config['SESSION_TYPE'] = 'filesystem'
app.config["SESSION_PERMANENT"] = False
Session(app)

In [262]:
##########################################################
# Server Functionalities
##########################################################
def shutdown_event():
    # Clean up resources
    print(f"[DEBUG] shutdown_event it called")
    session.clear()

# Register the shutdown function
atexit.register(shutdown_event)

def validate_token(token: str):
    """
    Validate the token here (e.g., check against a database or token provider)
    Return an HTTPException with appropriate status code if the token is invalid
    """
    try:
        result = is_valid_token(token)
        if result == False:
            raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid token")
        return token
    except Exception as e:
        raise HTTPException(status_code=status.HTTP_500_INTERNAL_SERVER_ERROR, detail="Token validation failed")

def is_valid_token(token: str):
    # Simulate token validation failure (e.g., connection error)
    if token == "invalid_token":
        return False
    return True

def generate_answer(data, qna: QnA(), key: str, token:str, current_session):
    question = data['question']
    history = [["", ""]]
    if 'history' in data:
        #print(f"[DEBUG] data['history'] type: {type(data['history'])} - len: {len(data['history'])} - data val: {data['history']}")
        if len(data['history']) >= 1 and data['history'][-1][1] != None:
            #print(f"[DEBUG] data['history']: {data['history']}")
            history = data['history']
            # Pre-process previous answer
            match = re.search(r'(.*?)<br>', history[-1][1], re.DOTALL)
            if match:
                history[-1][1] = match.group(1).strip()

    if "path" in data: path = data["path"]
    else: path = ""

    final_prompt, reference, wkproduct_link, guideline_links, guidelines, is_followup_sentence = qna.generate_prompt(path, question, prompt_for_common_query, prompt_for_specific_inquiry, history)
    completed_final_prompt = final_prompt
    completed_final_prompt = completed_final_prompt.replace('{prev_question}', history[-1][0])
    completed_final_prompt = completed_final_prompt.replace('{prev_answer}', history[-1][1])
    
    if is_followup_sentence != True: # If it is not classified by rules, use model to classify it again for assurance.
        is_followup_sentence = model_classify_question_1(question)

    if is_followup_sentence == True: # If this is an ongoing question, add previous answer to the prompt
        completed_final_prompt = prompt_for_followup_question
        completed_final_prompt = completed_final_prompt.replace('{prev_question}', history[-1][0])
        completed_final_prompt = completed_final_prompt.replace('{prev_answer}', history[-1][1])
        completed_final_prompt = completed_final_prompt.replace('{current_question}', question)

    print("############## COMPLETED FINAL PROMPT ##############")
    print(completed_final_prompt)

    related_question_prompt = " "
    if "Now read the following information pieces" in completed_final_prompt:
        related_question_prompt = qna.generate_related_question_prompt(question, relate_question_prompt, history)

    # Collect the latest answer from the reply
    current_reply = ""
    # Collect replies in a list. Reply can includes: answer + links + references
    accumulated_reply = []
    for reply in chatNew(completed_final_prompt, related_question_prompt, reference, wkproduct_link, guideline_links, guidelines):
        accumulated_reply.append(reply)
        #print(f"[DEBUG] reply: {reply}")
        if support_sessions[token]['stop_streaming'] == True:
            with support_sessions_lock:
                support_sessions[token]['stop_streaming'] = False
                return

        data = json.dumps({"status":1, "answer":reply, "key":key})
        yield f"data: {data}\n\n"
    
    clear_torch_cache()
    return

@app.route("/connect", methods=['POST'])
def connect():
    try:
        current_active_session = list(session.keys())
        raw_token = request.headers.get('Authorization')
        
        if not raw_token:
            print(f"[DEBUG] connect - Bad request")
            return {"message": "Token not provided"}, 400 # Badd request

        if raw_token.startswith('Bearer '):
            token_parts = raw_token.split()
            token = token_parts[1]
            if token not in session:
                # Initialize session data for the current user
                qna_instance = QnA()
                pickled_qna = pickle.dumps(qna_instance)
                session_data = {'token': token,
                                'qna_instance': pickled_qna,
                                'conversation': [["", ""]],
                                'stop_streaming': False
                               }
                session[token] = session_data
                
            return {"message": "Connected"}    
            
        else:
            return {"message": "Invalid token"}, 401 # Unauthorized
            
    except:
        print(f"[DEBUG] Connection Error")
        traceback.print_exc()
        return {"message": "Connection Error"}, 500 # Internal server error
        
@app.route("/disconnect", methods=['POST'])
def disconnect():
    try:
        raw_token = request.headers.get('Authorization')
        if not raw_token:
            print(f"[DEBUG] disconnect - Bad request")
            return {"message": "Token not provided"}, 400 # Badd request

        if raw_token.startswith('Bearer '):
            token_parts = raw_token.split()
            token = token_parts[1]
            # Deallocate session data for the current user
            if token in session:
                del session[token]

                # Reset stop_streaming flag
                with support_sessions_lock:
                    if token in support_sessions:
                        del support_sessions[token]      

            print(f"[DEBUG] Disconnect OK")

            return {"message": "Disconnected"}

        return {"message": "Invalid token"}, 401 # Unauthorized
        
    except:
        print(f"[DEBUG] Disconnect - Error")
        traceback.print_exc()
        return {"message": "Diconnection Error"}, 500 # Internal server error

@app.route('/qna', methods=['POST'])
def qna():
    try:
        raw_token = request.headers.get('Authorization')
        if not raw_token:
            print(f"[DEBUG] qna - Bad request")
            return {"message": "Token not provided"}, 400 # Badd request
            
        if raw_token.startswith('Bearer '):
            token_parts = raw_token.split()
            token = token_parts[1]

            data = request.get_json()
            pickled_qna = session[token].get('qna_instance')
            #print(f"[DEBUG] pickled_qna size: {asizeof.asizeof(pickled_qna)}")
            if not pickled_qna:
                print(f"[DEBUG] qna - load qna_instace failed")
                return json.dumps({'status': 0})
                
            qna_instance = pickle.loads(pickled_qna)
            #print(f"[DEBUG] qna_instance size: {asizeof.asizeof(qna_instance)}")
            classify_question_attribute = qna_instance.get_classify_question()
            #print(f"[DEBUG] qna_instance size: {asizeof.asizeof(qna_instance)}") 
            
            if validate_token(token):
                if "question" in data:
                    question = data["question"]
                    key = time.time()  # Create a unique key for each request
                    # Initialize the flag
                    with support_sessions_lock:
                        if token not in support_sessions:
                            support_sessions[token] = {'stop_streaming': False}
                        else:
                            support_sessions[token]['stop_streaming'] = False
                    #conversation = get_conversation() 
                    # Create an text generator
                    #generator = generate_answer(data, qna_instance, key, conversation, token)
                    #return Response(stream_with_context(generator), content_type="application/json")
                    #data = json.dumps({"status":1, "answer": ["This", "This is", "This is demo", "This is demo texts"], "key": key})
                    generator = generate_answer(data, qna_instance, key, token, session)
                    #return Response(generator, content_type="application/json")
                    return Response(generator, mimetype="text/event-stream")
                    #return data
            else:
                return json.dumps({"status":0}), 401 # Unauthorized
                
        else:
            return json.dumps({"status":0}), 401 # Unauthorized
            
    except:
        print(f"[DEBUG] qna route - Error")
        traceback.print_exc()
        return {"message": "qna route error"}, 500 # Internal server error

@app.route("/getQueueLength", methods=['POST'])
def getQueueLength():
    data = request.get_json()
    if semaphore is None:
        return json.dumps({"status":0})

    inQueue = semaphore._value if semaphore._value is not None else 0
    inWaiting = global_counter - MAX_CONCURRENCY + inQueue
    print(inWaiting, " - ", global_counter, " - ", inQueue)
    return json.dumps({"status":1, "in_queue": inQueue, "in_waiting":inWaiting, "max_concurrency":MAX_CONCURRENCY})

@app.route("/stopStreaming", methods=['POST'])
def stop_streaming():
    try:
        raw_token = request.headers.get('Authorization')
        if not raw_token:
            print(f"[DEBUG] stopStreaming - Bad request")
            return {"message": "Token not provided"}, 400 # Bad request
            
        if raw_token.startswith('Bearer '):
            token_parts = raw_token.split()
            token = token_parts[1]

            data = request.get_json()
            print("[DEBUG] Stop Streaming Is Called")
            if "key" in data:
                if token not in session:  
                    return {"message": "Invalid token"}, 401 # Unauthorized
                    
                with support_sessions_lock:
                    support_sessions[token]['stop_streaming'] = True   
                key = data["key"]
                
                return json.dumps({"status":1})
                
            return json.dumps({"status":0}), 401 # Unauthorized
                
        else:
            return json.dumps({"status":0}), 401 # Unauthorized
            
    except:
        print(f"[DEBUG] qna route - Error")
        traceback.print_exc()
        return {"message": "qna route error"}, 500 # Internal server error
            

@app.route("/uploadDocument", methods=['POST'])
def upload_document():
    print("[DEBUG] uploadDocument Is Called")
    data = request.get_json()
    if "path" in data and "data" in data and "file_name" in data:
        pickled_qna = session.get('qna_instance')
        print(f"[DEBUG] pickled_qna is created")
        if pickled_qna:
            qna_instance = pickle.loads(pickled_qna)
            print(f"[DEBUG] qna_instance is created")
            qna_instance.save_external_files(data["data"], data["path"], data["file_name"])
            return json.dumps({"status":1})
            
    return json.dumps({"status":0})

@app.route("/deleteDocument", methods=['POST'])
def delete_document():
    print("[DEBUG] deleteDocument Is Called")
    data = request.get_json()
    
    if "path" in data:
        pickled_qna = session.get('qna_instance')
        print(f"[DEBUG] pickled_qna is created")
        if pickled_qna:
            qna_instance = pickle.loads(pickled_qna)
            print(f"[DEBUG] qna_instance is created")
            qna_instance.delete_external_files(data["path"])
            return json.dumps({"status":1})
        
    return json.dumps({"status":0})

@app.route("/hello", methods=['GET'])
def hello():
    session['user_id'] = 'unique_id_for_each_client'
    print(f"\n[DEBUG] qna cookies: {request.cookies}")
    session.clear()
    
    return 'Hello World!'

Loading LLM model tokenizer ...

QA_init_time: 0.039073944091796875 seconds


## Running

In [263]:
from werkzeug.serving import run_simple

In [ ]:
##########################################################
# Server Main Function
##########################################################
clear_torch_cache() 
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=1235, debug=True, use_reloader=False)
    #run_simple("172.29.173.25", 1235, app)